In [1]:
import os
import pathlib
import tensorflow as tf

from tensorflow import keras as keras
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import MobileNetV2

In [2]:
NUM_CLASSES = 2

TRAIN_DATA_PATH = '/home/mist/mask_datasets/train/'
VALID_DATA_PATH = '/home/mist/mask_datasets/valid/'

BATCH_SIZE = 32
TRAIN_EPOCHS = 5

IMG_HEIGHT = 224
IMG_WIDTH = 224

train_datagen = ImageDataGenerator(
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)  # Set as training dataset

validation_generator = valid_datagen.flow_from_directory(
    VALID_DATA_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)  # Set as validation dataset

print(train_generator.class_indices)
print(validation_generator.class_indices)

Found 92366 images belonging to 2 classes.
Found 6882 images belonging to 2 classes.
{'face': 0, 'masked_face': 1}
{'face': 0, 'masked_face': 1}


In [3]:
import matplotlib.pyplot as plt

# Check for valid images
g = os.walk(TRAIN_DATA_PATH)
for path,dir_list,file_list in g:  
    for file_name in file_list:
        try:
            plt.imread(path + '/'+ file_name)
        except:
            print(path + "/" + file_name)
print("Check for train images")

g = os.walk(VALID_DATA_PATH)
for path,dir_list,file_list in g:  
    for file_name in file_list:
        try:
            plt.imread(path + '/'+ file_name)
        except:
            print(path + "/" + file_name)

print("Check for valid images")

Check for train images
Check for valid images


In [4]:
base_model = MobileNetV2(weights='imagenet',
                         include_top=False,
                         input_shape=[IMG_HEIGHT, IMG_WIDTH, 3],
                         classes=NUM_CLASSES)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
dense = keras.layers.Dense(1024, activation = 'relu')(avg)
dense = keras.layers.Dense(1024, activation = 'relu')(dense)
dense = keras.layers.Dense(512, activation = 'relu')(dense)
preds = keras.layers.Dense(NUM_CLASSES, activation = 'sigmoid')(dense)
model = keras.Model(inputs = base_model.input, outputs = preds)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3, 
    verbose=1)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=1e-4),
              metrics = ['binary_accuracy'])

model.fit(
    train_generator,
    epochs=TRAIN_EPOCHS,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size,
    callbacks=[reduce_lr])

model.save('mask_recongnition')

Epoch 1/5
2886/2886 [==============================] - 1154s 400ms/step - loss: 0.0096 - binary_accuracy: 0.9964 - val_loss: 0.0013 - val_binary_accuracy: 0.9991 - lr: 1.0000e-04
Epoch 2/5
2886/2886 [==============================] - 1156s 401ms/step - loss: 0.0046 - binary_accuracy: 0.9988 - val_loss: 0.0039 - val_binary_accuracy: 0.9989 - lr: 1.0000e-04
Epoch 3/5
2886/2886 [==============================] - 1144s 397ms/step - loss: 0.0042 - binary_accuracy: 0.9989 - val_loss: 0.0036 - val_binary_accuracy: 0.9994 - lr: 1.0000e-04
Epoch 4/5
2886/2886 [==============================] - ETA: 0s - loss: 0.0036 - binary_accuracy: 0.9991
Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
2886/2886 [==============================] - 1145s 397ms/step - loss: 0.0036 - binary_accuracy: 0.9991 - val_loss: 0.0016 - val_binary_accuracy: 0.9999 - lr: 1.0000e-04
Epoch 5/5
2886/2886 [==============================] - 1151s 399ms/step - loss: 0.0021 - binary_accuracy: 0.99